## Calculation of Time Intervals

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from qutip import *

In [3]:
# Time dependent JC simulation of synthesis sequence
doff = 2 * np.pi * (-463)    # MHz; off-resonance detuning
wr = 2 * np.pi * 6570        # MHz; resonator frequency
omq = 1000                   # MHz/2pi; qubit drive strength
om = 2 * np.pi * 19          # MHz; coupling strength
N = 4                        # number of cavity fock states

# states
ground = basis(2, 1)
excited = basis(2, 0)
vac = fock(N, 0)
one = fock(N, 1)
two = fock(N, 2)
three = fock(N, 3)

# target state = |g>(|1> + i|3>)
target = tensor(ground, (one + 1j*three).unit())
# initial state = |g>|0>
psi0 = tensor(ground, vac)

# operators
a = tensor(qeye(2), destroy(N))
sm = tensor(sigmam(), qeye(N))
sz = tensor(sigmaz(), qeye(N))

# hamiltonians
H_swap = 0.5 * om * (sm * a.dag() + sm.dag() * a)         # on-resonance swap
H_doff = doff * sm.dag() * sm           # off-resonance phase shift
H_pulse = 0.5 * omq * (sm.dag() + sm)  # off-resonance qubit drive

In [24]:
from numpy import pi, sqrt, cos, sin, arctan, exp

In [25]:
# t8: swap
arg = pi/(2*sqrt(3))
t8 = pi/(om * sqrt(3))
print('t8:',t8)
e2 = -1/sqrt(2)
g1 = 1/sqrt(2) * cos(pi / (2 * sqrt(3)))
e0 = 1j/sqrt(2) * sin(pi / (2 * sqrt(3)))
print('g1:', g1, '\ne0:', e0, '\ne2:', e2)

t8: 0.0151934281365691
g1: 0.43571248704868193 
e0: 0.556915279580164j 
e2: -0.7071067811865475


In [19]:
# t7: pulse
t7 = pi/omq
print('t7:', t7)
g0 = -1/sqrt(2) * sin(pi/(2*sqrt(3)))
g2 = -1j/sqrt(2)
e1 = 1j/sqrt(2) * cos(pi/(2*sqrt(3)))
print('g0:', g0, 'exp(i*doff*t7)', '\ng2:', g2, 'exp(i*doff*t7)', '\ne1:', e1)

t7: 0.0031415926535897933
g0: -0.556915279580164 exp(i*doff*t7) 
g2: (-0-0.7071067811865475j) exp(i*doff*t7) 
e1: 0.43571248704868193j


In [29]:
# t6: rotation
# same as above, but with added phase shift of exp(i*doff*t6) to |e,1> term
t6 = t7 - pi/(2*doff)
print('t6:', t6)
g0 = -exp(1j*doff*t7) * 1/sqrt(2) * sin(arg)
g2 = -1j * exp(1j*doff*t7) * 1/sqrt(2)
e1 = 1j/sqrt(2) * exp(1j*doff*t6) * cos(arg)
print('g0:', g0, '\ng2:', g2, '\ne1:', e1)

t6: 0.003681549457045517
g0: (0.5343680710788288+0.15686106349678972j) 
g2: (-0.1991640843223585+0.6784789366782409j) 
e1: (-0.41807228098443483-0.12272301838945959j)


In [31]:
# t5: swap
t5 = sqrt(2)/om * arctan(1 / cos(arg))
print('t5:', t5)
arg5 = om * t5 / sqrt(2)
argd6 = 1j*doff*t6
argd7 = 1j*doff*t7
e1 = 1j/sqrt(2) * exp(argd6) * cos(arg) * cos(arg5) + 1/sqrt(2) * exp(argd7) * sin(arg5)
g0 = -1/sqrt(2) * exp(argd7) * sin(arg)
print('e1:', e1, '\ng0:', g0)

t5: 0.01206609876842327
e1: (-0.796942971387266-0.23393860674669148j) 
g0: (0.5343680710788288+0.15686106349678972j)


In [36]:
# t4: pulse
t4 = pi/omq
argd4 = 1j*doff*t4
print('t4:', t4)
g1 = exp(argd4) * (-1/sqrt(2) * exp(argd6) * cos(arg) * cos(arg5) + 1j/sqrt(2) * exp(argd7) * sin(arg5))
e0 = -1j/sqrt(2) * exp(argd7) * sin(arg)
print('g1:', g1, '\ne0:', e0)
g1_t4 = g1
e0_t4 = e0 # for use in next section

t4: 0.0031415926535897933
g1: (-0.4489347899821952+0.6987867526747985j) 
e0: (-0.15686106349678972+0.5343680710788288j)


In [56]:
# t3: rotate
# swap condition:
x = (-1/sqrt(2) * exp(argd6) * cos(arg) * cos(arg5) + 1j/sqrt(2) * exp(argd7) * sin(arg5)) / (-exp(argd7) * 1j/sqrt(2) * sin(arg))
print('temp variable:', x)
t3 = t4-pi/(2*doff)
print('t3:', t3)
argd3 = 1j*doff*t3
g1 = exp(argd4) * (-1/sqrt(2) * exp(argd6) * cos(arg) * cos(arg5) + 1j/sqrt(2) * exp(argd7) * sin(arg5))
e0 = -exp(argd3) * 1j/sqrt(2) * exp(argd7) * sin(arg)
print('g1:', g1, '\ne0:', e0)

temp variable: (-1.4913746058560873-4.782031939657682e-17j)
t3: 0.003681549457045517
g1: (-0.4489347899821952+0.6987867526747985j) 
e0: (-0.4685521329992587-0.30102080873537146j)


In [61]:
# t2: swap
# ignore imaginary part of x, since it's so small
t2 = 2 / om * (arctan(-np.real(x)))
arg2 = om * t2 / 2
print('t2:', t2)
c1 = exp(argd3) * exp(argd7) * 1j/sqrt(2) * sin(arg) * cos(arg2)
c2 = 1j * exp(argd4) * (-1/sqrt(2) * exp(argd6) * cos(arg) * cos(arg5) + 1j/sqrt(2) * exp(argd7) * sin(arg5)) * sin(arg2)
e0 = c1 + c2
print('e0:', e0)
ce0_t2 = e0

t2: 0.01642025261264906
e0: (-0.319446985769817-0.2052283689095043j)


In [63]:
# t1: pulse
t1 = pi / omq
print('t1:', t1)
argd1 = 1j*doff*t1
g0 = 1j * exp(argd1) * ce0_t2
print('g0:', g0)

t1: 0.0031415926535897933
g0: (-0.2868951300168535+0.2487091564363684j)
